#Examen Parcial
##CC58 Tópicos en Ciencias de la Computación
###Pedro Shiguihara
####UPC 2021-1
https://colab.research.google.com/drive/1RNYPtkCOOlIVgzwcBu47QGYxXc2vvCg-?usp=sharing

---
Integrantes:
* Barrientos, Fiorella - u201711590
* Salazar, Antonio - u201714978
* Villarruel, Sergio - u20174083 

In [ ]:
%%writefile data.txt
3
2,2,3
2
0->1
1->2
1.0
10
0,0,0
0,0,1
0,1,0
1,0,0
0,1,2
0,1,2
1,1,0
0,1,0
1,0,1
1,1,1

Writing data.txt


In [ ]:
import ipywidgets as widgets
#https://colab.research.google.com/drive/1HnkyKNSvUqcsJkh5yGYCcZxBaCgxsQLl?usp=sharing

In [ ]:
class Dataframe:
    """
    CREA UN OBJETO DATAFRAME QUE IMPLEMENTA MÉTODOS DE FUNCIÓN DE CONTEO
    CARDINALIDAD, PROBABILIDAD MARGINAL, Y PROBABILIDAD CONJUNTA
    """
    def __init__(self,path):
        #LECTURA DE DATOS
        #RECOLECCION DE FILAS
        finput = open(path,"r")
        self.n_var = 0
        self.ca
        self.n_var = int(finput.readline().rstrip()) # Numero de variables
        self.card=[]
        self.headers=[]
        self.c = finput.readline().rstrip().split(',') # Cardinalidad
        for i in range(self.n_var):
          self.card.append(int(self.c[i]))
          self.headers.append(i)
        self.n_edges = int(finput.readline().rstrip()) # Numero de aristas
        self.edges = [[] for _ in range(self.n_var)]   # Aristas
        for _ in range(self.n_edges):
            left, right = finput.readline().rstrip().split('->')
            self.edges[int(right)].append(int(left)) # Leer aristas
        self.hiper_param = float(finput.readline().rstrip()) # Hiperparametro de dirichlet
        self.M = int(finput.readline().rstrip()) # Numero de instancias
        self.data = [[] for _ in range(self.M)] # Data
        for i in range(self.M):
          data_split = finput.readline().rstrip().split(',')
          for val in data_split:
              self.data[i].append(int(val))
        finput.close()
        self.cols = []
        # OBTENER VALORES DISTINTOS DE CADA COLUMNA
        self.unique_val = dict()
        for i, head in enumerate(self.headers):
            self.unique_val[head] = list()
            for row in self.data:
                if row[i] not in self.unique_val[head]:
                    self.unique_val[head].append(row[i])
    
    #IMPRIMIR DATASET
    def __str__(self):
        result = """"""
        for head in self.headers:
            result += '|{:>11}| '.format(head)
        result+="\n---------------------------------------------------------------------\n"
        for row in self.data:
            for val in row:
                result+='|{:>11}| '.format(val)
            result+="\n"
        return result

    # #FUNCIÓN CARDINALIDAD
    # def card(self, column):
    #     try:
    #         return len(self.unique_val[column])
    #     except:
    #         print("No se encuentra la columna")

    #FUNCIÓN CONTEO
    def rep(self, params):
        cnt = 0
        for row in self.data:
            ok = True
            for x in params:
              if(row[x[0]] != x[1]):
                ok = False
            if(ok):
              cnt+= 1
        return cnt

    def prob_general(self, params, factor):
        # (m[value] + p) / (M + (p * card(column))
        exp1 = self.rep(params) + self.hiper_param
        exp2 = self.M + (self.hiper_param * factor)
        return round(exp1/exp2,6)    

    def get_dag(self):
      size = 0
      self.dag = []
      indexx = 0
      while len(self.dag) < len(self.edges):
          for i, val in enumerate(self.edges):
            if not val and not self.dag:
              self.dag.append(i)
              break
            else:
              if val:
                if self.dag[indexx] == val[0]:
                  self.dag.append(i)
                  indexx += 1
                  break

    def get_query(self):
      self.get_dag()
      query = []
      for col in self.dag:
        query.append([col,0])
      return query

    def fact(self):
      factores=[]
      for i in range(self.n_var):
        factores.append(get_factor(self,i))
      return factores

In [ ]:
df = Dataframe("data.txt")
print(df)
print(df.edges)

AttributeError: ignored

In [ ]:
def normalize(arr):
  s = 0
  ans = []

  for x in arr:
    s+= x
  
  for x in arr:
    ans.append(x/s)
  
  return ans

In [ ]:
def get_factor(df, id):

  def add_column(M, col):
    for i in range(len(col)):
      M[i].append(col[i])

  cols = [id] + df.edges[id]

  uniqs = []
  cnt = []
  ids = []
  vuelta = []
  total = 1

  for x in cols:

      uniqs.append([i for i in range(df.card[x])])
      ids.append(total)
          
      total*= df.card[x]
      
      cnt.append(0)
      vuelta.append(0)

  D = []
  c_ans = []
  c_norm = []
  M = []

  for u in range(total):
      vals = []
      i = 0
      for x in cols:
          vals.append([x, uniqs[i][cnt[i]]])
          i+= 1
      
      M.append( [e[1] for e in vals] )

      exp1 = df.prob_general(vals, total)
      exp2 = df.prob_general(vals[1:], total/df.card[id])

      if(len(df.edges[id]) == 0):
        D.append(exp1)
      
      else:
        D.append(exp1/exp2)

      for j in range(len(cols)):
          vuelta[j]+= 1
          if(vuelta[j] == ids[j]):
              cnt[j] = (cnt[j] + 1)%len(uniqs[j])
              vuelta[j] = 0

      if(len(D) == df.card[id]):
        c_ans+= D
        c_norm+= normalize(D)
        D = []
        
  # Agregando columnas
  add_column(M, c_ans)
  add_column(M, c_norm)

  return M


In [ ]:
def titulos():
  titles_comb=[]
  for i in range(len(df.headers)):
    x = df.headers[i]
    s = "P(" + str(x)
    if (len(df.edges[x]) > 0):
      s += '|' 
      edge_list = df.edges[x]
      for j in range(len(edge_list)):
        if(j != 0):
          s+= ","
        s += str(edge_list[j])
    s+=")"
    titles_comb.append(s)
  return titles_comb

In [ ]:
def titulos_with_values(drops):
  titles_comb=[]
  for i in range(len(df.headers)):
    x = df.headers[i]
    s = "P(" + str(x) + "=" + str(drops[i].value)
    if (len(df.edges[x]) > 0):
      s += '|' 
      edge_list = df.edges[x]
      for j in range(len(edge_list)):
        if(j != 0):
          s+= ","
        s += str(edge_list[j]) + "=" + str(drops[j].value)
    s+=")"
    titles_comb.append(s)
  return titles_comb

In [ ]:
def hash_list(c1, c2):
  return "cols: " + str(c1) + " | vals: " + str(c2) 

def map_fact(df):
  map = dict()
  for i in range(df.n_var):
    cols = [i] + df.edges[i] 
    fact = get_factor(df,i)
    for x in fact:
      map[hash_list(cols, x[:-2])] = x[-1]

  return map

def get_val_fact(map, cols, vals):
  return map[hash_list(cols, vals)]

In [ ]:
map = map_fact(df)

In [ ]:
get_val_fact(map, [0], [1])

0.416667

In [ ]:
def get_mult(df, map, cols, vals):
  ans = []
  D = dict(zip(cols, vals))

  for x in cols:
    cols_ = [x] + df.edges[x]
    vals_ = [D[y] for y in cols_]

    aux = get_val_fact(map, cols_, vals_)
    ans.append(aux)
  
  return ans

In [ ]:
get_mult(df, map, [0,1,2] , [0,1,0])

[0.583333, 0.624999781250164, 0.4444444444444445]

In [ ]:
def mostrarr(r,etiqueta,factor3):
  hijo=list()
  labels=list()
  values=list()
  
  for i in range(len(etiqueta)):
    label = widgets.Button(description = (etiqueta[i]),layout=widgets.Layout(width="100%"))
    value = widgets.Button(description = str(factor3[i]),layout=widgets.Layout(width="100%"))
    labels.append(label)
    values.append(value)

  label = widgets.Button(description = str("Resultado"),layout=widgets.Layout(width="100%"))
  value = widgets.Button(description = str(r),layout=widgets.Layout(width="100%"))
  labels.append(label)
  values.append(value)
  
  label_box = widgets.VBox(labels)
  val_box = widgets.VBox(values)
  hijo.append(widgets.HBox([label_box,val_box]))
  children.append(hijo[0])
  box.children=children

# APLICACIÓN

In [ ]:
#@title Cargar archivo local
uploader = widgets.FileUpload(accept='.txt',multiple=False,description="Subir .txt")
display(uploader)

FileUpload(value={}, accept='.txt', description='Subir .txt')

In [ ]:
#@title Subiendo archivo
if uploader.value == {}:
  print("No hay ningun archivo subido.")
else:
  uploaded_filename = next(iter(uploader.value))
  content = uploader.value[uploaded_filename]['content']
  with open('input.txt', 'wb') as f: f.write(content)

from time import sleep
sleep(5)
print("Archivo subido")

No hay ningun archivo subido.
Archivo subido


In [ ]:
#@title Leyendo dataframe
df = Dataframe("input.txt")
print(df)

factores = df.fact()
titles_comb = titulos()

FileNotFoundError: ignored

In [ ]:
#@title Tabla de Factores
import re

children = list()

for i in range(len(titles_comb)):
  
  boxes = []
  col = 0
  for j in range(len(factores[i])):
    labels = list()
    for k in range(len(factores[i][j])):
      label = widgets.Button(description = str(factores[i][j][k]),layout=widgets.Layout(width="10%"))
      labels.append(label) 
    label_box = widgets.HBox(labels)
    boxes.append(label_box)

  titles_ = re.findall('[0-9]+',titles_comb[i])
  
  title_labels = []
  for title_label in titles_:
    label = widgets.Button(description = title_label,layout=widgets.Layout(width="10%"))
    title_labels.append(label)
  title_labels.append(widgets.Button(description = "Prob",layout=widgets.Layout(width="10%")))
  title_labels.append(widgets.Button(description = "Prob Normalizada",layout=widgets.Layout(width="10%")))
  title_box = widgets.HBox(title_labels)
  children.append(widgets.VBox([title_box,*boxes]))

tab = widgets.Tab()
tab.children = children
for i in range(len(titles_comb)):
  tab.set_title(i,titles_comb[i])
tab

In [ ]:
#@title Formulario para el cálculo de probabilidad
map = map_fact(df)
children = list()
box=widgets.VBox()
drops=list()
global borrar
borrar =False
for i in df.headers:
  
  drop = widgets.Dropdown(options=df.unique_val[i],
    value=df.unique_val[i][0],
    description="Variable {}:".format(i),
    )
  # drop.observe(on_change)
  drops.append(drop)
children.append(widgets.VBox(drops))
calcular=widgets.Button(
    description='Calcular',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
    )

def Click(b):
    etiqueta=[]
    factor3=[]
    resultado=0
    global borrar
    b=True
    if borrar:
      children.pop()
      #children[-1].close()
    query = []
    for i,_ in enumerate(drops):
      query.append(_.value)

    result = 1
    titles_comb_with_values = titulos_with_values(drops)
    for i, val in enumerate(get_mult(df,map,df.headers,query)):

      etiqueta.append(titles_comb_with_values[i])
      factor3.append(val)
      result *= val
    
    borrar=b
    resultado=result
    mostrarr(resultado,etiqueta,factor3)

children.append(calcular)
calcular.on_click(Click)

box.children=children
box

In [ ]:
import ipywidgets as widgets
from IPython.display import display

upl_dag = widgets.FileUpload(accept='.txt',multiple=False,description="Select dag.txt")
upl_data = widgets.FileUpload(accept='.csv',multiple=False,description="Select data.csv")


def upl_dag_file(a):
  global upl_dag
  if upl_dag.value == {}:
    print("No hay ningun archivo subido.")
  else:
    uploaded_filename = next(iter(upl_dag.value))
    content = upl_dag.value[uploaded_filename]['content']
    with open('dag.txt', 'wb') as f: f.write(content)
    print("Archivo subido")
    upl_dag.value.clear()
    upl_dag._counter = 0

def upl_data_file(a):
  global upl_data
  if upl_data.value == {}:
    print("No hay ningun archivo subido.")
  else:
    uploaded_filename = next(iter(upl_data.value))
    content = upl_data.value[uploaded_filename]['content']
    with open('data.csv', 'wb') as f: f.write(content)
    print("Archivo subido")
    upl_data.value.clear()
    upl_data._counter = 0

btn_upl_dag = widgets.Button(
    description='Upload DAG',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
    )

btn_upl_data = widgets.Button(
    description='Upload DATASET',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
    )

btn_upl_dag.on_click(upl_dag_file)
btn_upl_data.on_click(upl_data_file)

upl_dag_box = widgets.HBox([upl_dag,btn_upl_dag])
upl_data_box = widgets.HBox([upl_data,btn_upl_data])
upl_box =  widgets.VBox([upl_data_box,upl_dag_box])
display(upl_box)

hiperparam = widgets.FloatSlider(description="α")
display(hiperparam)

left_aris = widgets.Text(
    value='forecast',
    placeholder='Type something',
    description='BEGIN:',
    disabled=False
)

right_aris = widgets.Text(
    value='forecast',
    placeholder='Type something',
    description='END:',
    disabled=False
)

ctrl_ins = widgets.Button(
    description='ADD',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
    )

ctrl_del = widgets.Button(
    description='DELETE',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
    )

ctrl_show = widgets.Button(
    description='SHOW',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
    )

aris_param_box = widgets.HBox([left_aris,right_aris])
aris_ctrl_box = widgets.HBox([ctrl_ins,ctrl_del,ctrl_show])
aris_box = widgets.VBox([aris_param_box,aris_ctrl_box])
display(aris_box)


FloatSlider(value=0.0, description='α')

In [ ]:
def readDAG(path):
    finput = open(path,"r")
    lines = finput.readlines()
    finput.close()
    dag_len = len(lines)
    # Check dag_len
    edges = [[] for _ in range(dag_len)]
    for i in range(dag_len):
        line = lines[i].rstrip().split(',')
        for j in range(len(line)):
            if int(line[j]) == 1:
                edges[j].append(i)
    return edges

In [ ]:
# def readDAGMatrix(path):
#     finput = open(path,"r")
#     lines = finput.readlines()
#     dag_len = len(lines)
#     # Check dag_len
#     edges = [[0 for x in range(dag_len)] for _ in range(dag_len)]
#     for i in range(dag_len):
#         line = lines[i].rstrip().split(',')
#         for j in range(len(line)):
#             if int(line[j]) == 1:
#                 edges[j][i] = 1
#     return edges

In [ ]:
edges = readDAG("dag.txt")
edges

FileNotFoundError: ignored

In [ ]:
def deleteEdge(edges, left, right): # left -> right // 2->0
    index = edges[right].index(left)
    edges[right].pop(index)
    print(edges)

In [ ]:
deleteEdge(edges, 2, 0)

In [ ]:
def addEdge(edges, left, right): # left -> right // 2->0
    edges[right].append(left)
    print(edges)

In [ ]:
addEdge(edges, 2, 0)

In [ ]:
pip install graphviz

In [ ]:
from graphviz import Digraph

In [ ]:
def showGraph(edges):
    dot = Digraph()
    for _ in range(len(edges)):
        dot.node(str(_))
    for i in range(len(edges)):
        edge = edges[i]
        for j in range(len(edge)):
            dot.edge(str(edge[j]), str(i))
    dot.render("graph.gv")

In [ ]:
showGraph(edges)
with open("graph.gv") as f:
    dot_graph = f.read()
Source(dot_graph).pipe

NameError: ignored

In [ ]:
def readData(self, path):
    finput = open(path,"r")
    lines = finput.readlines()
    finput.close()
    self.M = len(lines)-1
    self.headers = lines[0].rstrip().split(',')
    self.n_var = len(self.headers)
    self.data = [[] for _ in range(self.M)]
    for i in range(self.M):
        data[i] = lines[i+1].rstrip().split(',')
    self.cols = []
    self.unique_val = dict()
    for i, head in enumerate(headers):
        unique_val[head] = list()
        for row in data:
            if row[i] not in unique_val[head]:
                unique_val[head].append(row[i])

In [ ]:
readData("data.csv")

In [ ]:
hiper_param = 1

In [ ]:
hiper_param_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
hiper_param_slider

In [ ]:
hiper_param = hiper_param_slider.value
hiper_param